Installing required dependencies

In [1]:
! pip install "gql[all]"

You should consider upgrading via the 'C:\Users\muffi\Development\gh-api-search\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


Setting up the gql library

In [2]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
from os import environ

bearer_token = environ.get("API_KEY")
headers = {
  "Authorization": f"bearer {bearer_token}"
}
transport = AIOHTTPTransport(url="https://api.github.com/graphql", headers=headers)

client = Client(transport=transport, fetch_schema_from_transport=True)

In [3]:
query = gql(
    """
query Search($query:String!, $first:Int! $cursor:String) {
    math: search(
        query: $query
        type: REPOSITORY
        first: $first
        after: $cursor
    ) {
        repositoryCount
        edges {
            node {
                ... on Repository {
                    nameWithOwner
                    url
                    homepageUrl
                    primaryLanguage {
                        name
                    }
                    forks {
                        totalCount
                    }
                    stargazers {
                        totalCount
                    }
                    description
                    readme_master: object(expression: "master:README.md") {
                        ... on Blob {
                            text
                        }
                    }
                    readme_main: object(expression: "main:README.md") {
                        ... on Blob {
                            text
                        }
                    }
                    defaultBranchRef {
                        target {
                            ... on Commit {
                                tree {
                                    files: entries {
                                        name
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        pageInfo {
            endCursor
            hasNextPage
        }
    }
    rateLimit {
        limit
        cost
        remaining
        resetAt
    }
}

"""
)

In [4]:
params = {
  "query": "applied math doi in:readme,description created:2023",
  "first": 50
}
try:
  result = await client.execute_async(query, variable_values=params)
except TimeoutError as e:
  print(e)  

TimeoutError: 

In [ ]:
from json import dumps
print(dumps(result))